In [1]:
import librosa
import sklearn
import pandas as pd
import time
import os

In [2]:
### 초기 설정

ROOT = "./data/Dataset"    # 데이터셋이 존재하는 폴더의 경로 지정
LIMIT_CNT = 70    # 분석할 음악 개수(None: "제한없음" / 50:" 장르별 50곡 제한")

In [3]:
### 속성 추출 함수

def extract_music_prop(filename = None, genre = None):
    path = f"{ROOT}/{genre}/{filename}"
    audio_info = [None for _ in range(len(audio_header))]
    y, sr = librosa.load(path)
    y, _ = librosa.effects.trim(y=y)
    length = int(len(y) / sr)
    chromagram = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=512)
    rmsTest = librosa.feature.rms(y=y)
    spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]
    rolloffTest = librosa.feature.spectral_rolloff(y=y, sr=sr)
    rolloffTest.mean()
    rolloffTest.var()
    zero_crossings = librosa.zero_crossings(y=y, pad=False)
    y_harm, y_perc = librosa.effects.hpss(y=y)
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    
    audio_info[0] = filename
    audio_info[1] = length
    audio_info[2] = chromagram.mean() 
    audio_info[3] = chromagram.var()
    audio_info[4] = rmsTest.mean()
    audio_info[5] = rmsTest.var()
    audio_info[6] = spectral_centroids.mean()
    audio_info[7] = spectral_centroids.var()
    audio_info[8] = spectral_bandwidth.mean()
    audio_info[9] = spectral_bandwidth.var()
    audio_info[10] = rolloffTest.mean()
    audio_info[11] = rolloffTest.var()
    audio_info[12] = zero_crossings.mean()
    audio_info[13] = zero_crossings.var()
    audio_info[14] = y_harm.mean()
    audio_info[15] = y_harm.var()
    audio_info[16] = y_perc.mean()
    audio_info[17] = y_perc.var()
    audio_info[18] = tempo

    def normalize(x, axis=0):
        return sklearn.preprocessing.minmax_scale(x, axis=axis)
    mfccs = librosa.feature.mfcc(y=y, sr=sr)
    mfccs = normalize(mfccs, axis=1)

    for i in range(19, 59, 2):
        idx = int((i - 18) / 2)
        audio_info[i] = mfccs[idx].mean()
        audio_info[1 + i] = mfccs[idx].var()
        
    audio_info[59] = genre
    return audio_info

In [4]:
### 음악에서 추출할 속성명(csv파일 헤더)
# 데이터프레임 생성

audio_header = ["filename", "length", "chroma_stft_mean", "chroma_stft_var", "rms_mean", "rms_var",
                "spectral_centroid_mean", "spectral_centroid_var", "spectral_bandwidth_mean", "spectral_bandwidth_var",
                "rolloff_mean", "rolloff_var", "zero_crossing_rate_mean", "zero_crossing_rate_var", "harmony_mean",
                "harmony_var", "perceptr_mean", "perceptr_var", "tempo", "mfcc1_mean", "mfcc1_var", "mfcc2_mean",
                "mfcc2_var", "mfcc3_mean", "mfcc3_var", "mfcc4_mean", "mfcc4_var", "mfcc5_mean", "mfcc5_var",
                "mfcc6_mean", "mfcc6_var", "mfcc7_mean", "mfcc7_var", "mfcc8_mean", "mfcc8_var", "mfcc9_mean",
                "mfcc9_var", "mfcc10_mean", "mfcc10_var", "mfcc11_mean", "mfcc 11_var", "mfcc12_mean", "mfcc12_var",
                "mfcc13_mean", "mfcc13_var", "mfcc14_mean", "mfcc14_var", "mfcc15_mean", "mfcc15_var", "mfcc16_mean",
                "mfcc16_var", "mfcc17_mean", "mfcc17_var", "mfcc18_mean", "mfcc18_var", "mfcc19_mean", "mfcc19_var",
                "mfcc20_mean", "mfcc20_var", "label"]
audio_df = pd.DataFrame([], audio_header).T

In [5]:
### 폴더 구조 파악
# "./data/Dataset"에 만들어진 폴더명으로 장르 구분
# 폴더내의 wav 파일명 가져오기

file_genre_list = os.listdir(ROOT)
genre_dict = {}

for genre in file_genre_list:
    file_name_list = os.listdir(f"./data/Dataset/{genre}")
    genre_dict[genre] = file_name_list

In [ ]:
### 음악 분석 시작

total_start_time = time.time()
print("[Log] 음악 분석 시작...")
for genre in genre_dict:
    for i, filename in enumerate(genre_dict[genre]):
        if LIMIT_CNT and LIMIT_CNT <= i: continue
        start_time = time.time()
        try:
            audio_info = extract_music_prop(filename, genre)
            audio_df.loc[filename] = audio_info
            print(f"[Log] {genre} {i + 1}번째 곡 소요시간:{time.time() - start_time: .1f}s")
        except:
            print("[Error] {genre} {i + 1}번째 곡 분석 실패")

print(f"[LOG] 전체 분석 소요시간:{time.time() - total_start_time: .1f}s")

[Log] 음악 분석 시작...
[Log] ballad 1번째 곡 소요시간: 16.1s
[Log] ballad 2번째 곡 소요시간: 14.2s
[Log] ballad 3번째 곡 소요시간: 14.7s
[Log] ballad 4번째 곡 소요시간: 14.3s
[Log] ballad 5번째 곡 소요시간: 14.5s
[Log] ballad 6번째 곡 소요시간: 17.6s
[Log] ballad 7번째 곡 소요시간: 13.8s
[Log] ballad 8번째 곡 소요시간: 14.2s
[Log] ballad 9번째 곡 소요시간: 15.4s
[Log] ballad 10번째 곡 소요시간: 13.2s
[Log] ballad 11번째 곡 소요시간: 13.4s
[Log] ballad 12번째 곡 소요시간: 14.7s
[Log] ballad 13번째 곡 소요시간: 10.8s
[Log] ballad 14번째 곡 소요시간: 18.7s
[Log] ballad 15번째 곡 소요시간: 13.1s
[Log] ballad 16번째 곡 소요시간: 13.7s
[Log] ballad 17번째 곡 소요시간: 13.7s
[Log] ballad 18번째 곡 소요시간: 18.7s
[Log] ballad 19번째 곡 소요시간: 14.3s
[Log] ballad 20번째 곡 소요시간: 15.5s
[Log] ballad 21번째 곡 소요시간: 18.0s
[Log] ballad 22번째 곡 소요시간: 12.0s
[Log] ballad 23번째 곡 소요시간: 15.8s
[Log] ballad 24번째 곡 소요시간: 13.0s
[Log] ballad 25번째 곡 소요시간: 14.5s
[Log] ballad 26번째 곡 소요시간: 12.8s
[Log] ballad 27번째 곡 소요시간: 14.4s
[Log] ballad 28번째 곡 소요시간: 13.9s
[Log] ballad 29번째 곡 소요시간: 16.5s
[Log] ballad 30번째 곡 소요시간: 12.7s
[Log] ballad 31번째 곡 소요시간: 15.5s

In [ ]:
# 분석 결과 확인

audio_df

In [ ]:
# CSV 생성

audio_df.set_index(keys=['filename'], inplace=True, drop=True)
audio_df.to_csv("extract_music_prop_" + str(int(time.time())) + ".csv")